Language modeling

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy

C:\Users\ac32\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
'''
The large movie view dataset contains a collection of 50,000 reviews from IMDB.The dataset contains an even number of
positive and negative reviews.The authors considered only highly polarized reviews. A negative review has a score ≤ 4
out of 10, and a positive review has a score ≥ 7 out of 10. Neutral reviews are not included in the dataset. The dataset
is divided into training and test sets. The training set is the same 25,000 labeled reviews.

The sentiment classification task consists of predicting the polarity (positive or negative) of a given text.
'''


'\nThe large movie view dataset contains a collection of 50,000 reviews from IMDB.The dataset contains an even number of\npositive and negative reviews.The authors considered only highly polarized reviews. A negative review has a score ≤ 4\nout of 10, and a positive review has a score ≥ 7 out of 10. Neutral reviews are not included in the dataset. The dataset\nis divided into training and test sets. The training set is the same 25,000 labeled reviews.\n\nThe sentiment classification task consists of predicting the polarity (positive or negative) of a given text.\n'

In [4]:
PATH = 'C:/input/Fast_AI/aclImdb/'
TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'
TRN = '{}{}'.format(PATH,TRN_PATH)
VAL = '{}{}'.format(PATH,VAL_PATH)
!ls {PATH}

README
aclImdb.tar
imdb.vocab
imdbEr.txt
test
train


In [5]:
trn_files = !ls {TRN}
trn_files[:10]

['0_0.txt',
 '0_3.txt',
 '0_9.txt',
 '10000_0.txt',
 '10000_4.txt',
 '10000_8.txt',
 '10001_0.txt',
 '10001_10.txt',
 '10001_4.txt',
 '10002_0.txt']

In [6]:
review = !cat {TRN}{trn_files[6]}
review[0]

"Everybody has seen 'Back To The Future,' right? Whether you LIKE that movie or not, you've seen an example of how to make a time-travel movie work. A torn-up poster for 'Back To The Future' shows up in this movie, representing, perhaps unintentionally, what the makers of 'Tangents' (aka 'Time Chasers') did to the time-travel formula. Then again, the movie claims to have been made in 1994, but it looks -- and sounds -- like it was produced at least ten years earlier, so maybe they achieved time-travel after all.<br /><br />Start with an intensely unappealing leading man. I mean, what woman doesn't love gangly, whiny, lantern-jawed, butt-chinned, mullet-men with Coke-bottle glasses? Oh, none of you? Prepare to tough it out, ladies, cuz that's what this movie gives you.<br /><br />Second, add a leading lady who -- while not entirely unattractive -- represents many '80s clichÃ©s: big hair, too much makeup, two different plaids, shoulder pads, acid-washed mom-jeans, etc.<br /><br />Throw i

In [7]:
# 在linux下面找每个txt文件，再xargs cat查看，然后计算多少行
!find {TRN} -name '*.txt' | xargs cat|wc -w

0


File not found - '*.txt'


In [8]:
!find {VAL} -name '*.txt' | xargs cat|wc -w

0


File not found - '*.txt'


In [9]:
# tokenize into an array of words
#python -m spacy download en  #先要安装Spacy language model
#You can now load the model via spacy.load('en_core_web_sm')
spacy_tok = spacy.load('en')

In [10]:
# 分词，每个单词转为字符类型，
' '.join([sent.string.strip() for sent in spacy_tok(review[0])])

"Everybody has seen ' Back To The Future , ' right ? Whether you LIKE that movie or not , you 've seen an example of how to make a time - travel movie work . A torn - up poster for ' Back To The Future ' shows up in this movie , representing , perhaps unintentionally , what the makers of ' Tangents ' ( aka ' Time Chasers ' ) did to the time - travel formula . Then again , the movie claims to have been made in 1994 , but it looks -- and sounds -- like it was produced at least ten years earlier , so maybe they achieved time - travel after all.<br /><br />Start with an intensely unappealing leading man . I mean , what woman does n't love gangly , whiny , lantern - jawed , butt - chinned , mullet - men with Coke - bottle glasses ? Oh , none of you ? Prepare to tough it out , ladies , cuz that 's what this movie gives you.<br /><br />Second , add a leading lady who -- while not entirely unattractive -- represents many ' 80s clichÃ © s : big hair , too much makeup , two different plaids , sh

In [11]:
# 使用Pytorch的torchtext库预处理数据，spacy库tokenization
# 创建torchtext field, 描述怎么preprocessy一个文本，
# en = 'en_core_web_sm'
TEXT = data.Field(lower=True, tokenize='spacy') # 全部小写，使用spacy库tokenize


In [12]:
# bptt this define how many words are processing at a time in each row of the mini-batch. More importantly, 
# it defines how many 'layers' we will backprop through. Making this number higher will increase time and memory requirements, 
# but will improve the model's ability to handle long sentences.
bs=64; bptt=70

In [13]:
?LanguageModelData

In [15]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

RuntimeError: Device index must not be negative